In [ ]:
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel

strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"],
                                          cross_device_ops=tf.distribute.ReductionToOneDevice())
with strategy.scope():
    tokenizer = AutoTokenizer.from_pretrained("E:\\hugging_face\\bert-base-chinese")
    labels = []
    token_list = []
    with open('../data/ChnSentiCorp.txt', mode="r", encoding="UTF-8") as file:
        for line in tqdm(file.readlines()):
            line = line.strip().split(",")
            labels.append(int(line[0]))
            text = line[1]
            token = tokenizer.encode(text, truncation=True)
            token = token[:128] + [0] * (128 - len(token))
            token_list.append(token)
    labels = np.array(labels)
    token_list = np.array(token_list)
    model = TFAutoModel.from_pretrained("E:\\hugging_face\\bert-base-chinese")
    input_token = tf.keras.Input(shape=(128,), dtype=tf.int32, name="input1")
    embedding = model(input_token)[0]
    embedding = tf.keras.layers.Flatten()(embedding)
    output = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid, name="output")(embedding)
    model = tf.keras.Model(inputs=input_token, outputs=output)

    model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), #loss='binary_crossentropy',
                  loss=tf.keras.losses.binary_focal_crossentropy,
                  metrics=['accuracy'])

    model.fit(token_list, labels, batch_size=16, epochs=10)



INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


100%|██████████| 7765/7765 [00:01<00:00, 4593.82it/s]
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Epoch 1/10
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task

In [8]:
token1 = tokenizer.encode("不是不好,但可以用", truncation=True)
token1 = token1[:128] + [0] * (128 - len(token1))
model.predict([token1])

1/1 [==============================] - 1s 582ms/step


array([[0.2173224]], dtype=float32)